In [ ]:
# Initialize drive|
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Move to Google Drive 
%cd drive
%cd 'My Drive'
%cd 'MSc Stats Dissertation'

In [ ]:
## Install necessary libraries
!pip install deepsmiles
!pip install selfies
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

In [ ]:
## Go to correct place  in drive to allow us 
## to import libraries
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
## Import Necessary lIbraries 
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras as keras
import pandas as pd
import math
import tensorflow.keras.layers as layers
import time
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import matthews_corrcoef
import deepsmiles
from selfies import encoder, decoder  
import rdkit
import Utils.generate_utils as generate_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
## Converter to convert SMILES to Deep SMILES
converter = deepsmiles.Converter(rings = True, branches = True)

In [ ]:
## Set to correct float type for consistency with training
tf.keras.backend.set_floatx('float32')

In [ ]:
## Import data files
SELFIES = False
DEEP = False
train_smiles_path = './Datasets/train_Tox_data.smi'
test_smiles_path = './Datasets/AID_1189_datatable_all.csv'
actual_test_smiles_path = './Datasets/3643044675069416146.txt'
if SELFIES:
  vocab =np.load('./vocab/selfies_vocab.npy',allow_pickle=True)
  vocab_index = np.load('./vocab/selfies_vocab_index.npy',allow_pickle=True)
elif DEEP:
  vocab =np.load('./vocab/deep_vocab.npy',allow_pickle=True)
  vocab_index = np.load('./vocab/deep_vocab_index.npy',allow_pickle=True)
else:
  vocab =np.load('./vocab/vocab.npy',allow_pickle=True)
  vocab_index = np.load('./vocab/vocab_index.npy',allow_pickle=True)
vocab = dict(vocab.ravel()[0])
vocab_index = dict(vocab_index.ravel()[0])
smiles_train =  pd.read_csv(train_smiles_path,delimiter='\t',header=None)

MIN = 3027

In [ ]:
## Load in file and process it for later predcitions
smiles_test =  pd.read_csv(test_smiles_path,delimiter=',')
actual_test_smiles = pd.read_csv(actual_test_smiles_path,delimiter='\t',header = None)
train =[]
test = []
train_Y = []
test_Y = []
for index in range(len(smiles_train[1])):
  if smiles_train[1][index][:2] =='DB':
    test.append(index)
    test_Y.append(0)
  elif smiles_train[1][index][:1] =='D':
    train.append(index)
    train_Y.append(0)
  elif smiles_train[1][index][:2] =='T3':
    test.append(index)
    test_Y.append(1)
  else:
    train.append(index)
    train_Y.append(1)

train_X = smiles_train[0][train]
test_X = smiles_train[0][test]

In [ ]:
## Neccesary CONSTANTS
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EPOCHS = 10
LEARNING_RATE =  0.000312087049936
if SELFIES or DEEP:
  PAD_LEN = 250
else:
  PAD_LEN = 160 ## Maximum size of a SMILE (100 + BOS, EOS)
  print('HERE')
MAX_LEN = PAD_LEN 
DROP_OUT= 0.2
EMBEDDING_DIM = 192  ## Embedding dim of the characters
HIDDEN_DIM = 256
DROPOUT = 0.2
TRAIN = False
LATENT_DIM = 64
TRANSFORMER_DECODE = True


In [ ]:
## Integer encode for selfies
def integer_encode_selfies(selfies,vocab_dict):
  selfies_enc = []
  for char in selfies:
    try:
      selfies_enc.append(vocab_dict[char])
    except:
      return None
  return selfies_enc

In [ ]:
## Splits the selfies <molecule> into a list of character strings.
def split_selfie(molecule):
  return re.findall(r'\[.*?\]|\.', molecule)

## Takes processed selfies smiles and returns the tokenized 
## versions of the selfies
def tokenize_selfies(selfies):
  char_list = split_selfie(selfies)
  tokenized= []
  tokenized.append('<BOS>')
  i = 0 
  while i < len(char_list):
    char = char_list[i]
    tokenized.append(char)
    i = i+1
  tokenized.append('<EOS>')
  return tokenized

In [ ]:
import re
## replace Br and Cl with single letters
def replace_halogens(string):
  br = re.compile('Br')
  cl = re.compile('Cl')
  string = br.sub('R', string)
  string = cl.sub('L', string)
  return string

In [ ]:
## Takes processed smiles/deep smiles and returns the tokenized 
## versions of the smiles or deep semiles
## Note: Run replace halogens and replace percentages
## before running this method 
def tokenize_smiles(smiles):
  char_list = list(smiles)
  tokenized= []
  tokenized.append('<BOS>')
  i = 0 
  while i < len(char_list):
    char = char_list[i]
    tokenized.append(char)
    i= i+1
  tokenized.append('<EOS>')
  return tokenized

In [ ]:
## Integer encode fore SMILES and DeepSMILES
def integer_encode(smiles,vocab_dict):
  smiles_enc = []
  for char in smiles:
    if char in vocab:
      smiles_enc.append(vocab_dict[char])
    else:
       return None
  return smiles_enc

In [ ]:
## Process for later prediction
smile_pair_tokens = []
indexes = []
index = 0 
for smiles in train_X:
  if SELFIES:
    smiles = encoder(smiles)
    if smiles is not None:
      indexes.append(index)
      smile_pair_tokens.append(tokenize_selfies(smiles))
  elif DEEP:
    smiles = replace_halogens(smiles)
    smile_pair_tokens.append(tokenize_smiles(converter.encode(smiles)))
  else:
    smiles = replace_halogens(smiles)
    smile_pair_tokens.append(tokenize_smiles(smiles))
  index = index+1
smile_pair_tokens = np.array(smile_pair_tokens)
if SELFIES:
  train_Y = np.array(train_Y)[indexes]


smiles_ordered = tf.keras.preprocessing.sequence.pad_sequences(smiles_ordered,maxlen = PAD_LEN,padding='post')
tox_smiles = np.array(smiles_ordered)

In [ ]:
## Prrocess for later prediction 
smiles_ordered = []
indexes = []
index = 0 
for smiles in smile_pair_tokens:
  if SELFIES:
    encoded_smile = integer_encode_selfies(smiles,vocab)
    if encoded_smile is not None:
      smiles_ordered.append(encoded_smile)
    else:
      smiles_ordered.append(None)
  else:
    smiles_ordered.append(integer_encode(smiles,vocab))
l=[i for i,v in enumerate(smiles_ordered) if v != None ]
smiles_ordered = np.array(smiles_ordered)[l]
train_Y  = np.array(train_Y)[l]
l=[i for i,v in enumerate(smiles_ordered) if len(v) <MAX_LEN]
smiles_ordered = np.array(smiles_ordered)[l]
train_Y  = train_Y[l]



smile_pair_tokens = []
indexes = []
index = 0 
for smiles in test_X:
  if SELFIES:
    smiles = encoder(smiles)
    if smiles is not None:
      indexes.append(index)
      smile_pair_tokens.append(tokenize_selfies(smiles))
  elif DEEP:
    smiles = replace_halogens(smiles)
    smile_pair_tokens.append(tokenize_smiles(converter.encode(smiles)))
  else:
    smiles = replace_halogens(smiles)
    smile_pair_tokens.append(tokenize_smiles(smiles))
  index = index+1
smile_pair_tokens = np.array(smile_pair_tokens)
if SELFIES:
  test_Y = np.array(test_Y)[indexes]


smiles_ordered = []
for smiles in smile_pair_tokens:
  if SELFIES:
    encoded_smile = integer_encode_selfies(smiles,vocab)
    smiles_ordered.append(encoded_smile)
  else:
    smiles_ordered.append(integer_encode(smiles,vocab))
l=[i for i,v in enumerate(smiles_ordered) if v != None ]
smiles_ordered = np.array(smiles_ordered)[l]
test_Y  = np.array(test_Y)[l]
l=[i for i,v in enumerate(smiles_ordered) if len(v) <MAX_LEN]
cancer_smiles = np.array(smiles_ordered)[l]
test_Y  = test_Y[l]

In [ ]:
import GANS.renewed_smiles_vae as conv_smiles_vae
import GANS.ic50vae as ic50vae

## Import in the VAE so that embeddings of different compounds can be calculated
if IC50:
  smile_vae = ic50vae.SMILE_VAE(vocab_size= VOCAB_SIZE,embedding_dim=EMBEDDING_DIM, max_len= MAX_LEN,
                      latent_dim = LATENT_DIM, recurrent_dropout = DROP_OUT,dropout_rate= DROP_OUT)
  if DEEP:
    print('IC50 DEEP')
    smile_vae.load_weights('ic50g_deep_conv_vae_weights')
  elif SELFIES:
    print('IC50 SELFIES')
    smile_vae.load_weights('ic50g_selfies_conv_vae_weights')
  else:
    print('IC50 NORMAL')
    smile_vae.load_weights('ic50g_smiles_conv_vae_weights')
else:
  smile_vae = conv_smiles_vae.SMILE_VAE(vocab_size= VOCAB_SIZE,embedding_dim=EMBEDDING_DIM, max_len= MAX_LEN, 
                      latent_dim = LATENT_DIM, recurrent_dropout = DROP_OUT,dropout_rate= DROP_OUT)
  if DEEP:
    print('DEEP')
    smile_vae.load_weights('deep_conv_vae_weights2')
  elif SELFIES:
    print('SELFIES')
    smile_vae.load_weights('selfies_conv_vae_weights2')
  else:
    print('NORMAL')
    smile_vae.load_weights('smiles_conv_vae_weights2')
 

In [ ]:
## Get the latents representations for the training data
train_latents = []
index = 0 
for smile in smiles_ordered:
  train_latents.append(smile_vae.encoder(smile.reshape(1,MAX_LEN))[1])
  if index %1000 == 0:
    print(index)
  index+=1

In [ ]:
## Get test representations for the test data 
cancer_smiles = tf.keras.preprocessing.sequence.pad_sequences(cancer_smiles,maxlen = PAD_LEN,padding='post')
cancer_smiles = np.array(cancer_smiles)
test_latents = []
index = 0 
for smile in cancer_smiles:
  test_latents.append(smile_vae.encoder(smile.reshape(1,MAX_LEN))[1])
  if index %100 == 0:
    print(index)
  index+=1
test_latents = np.array(test_latents)

In [ ]:
def train_model(model,train_X,train_Y,test_X,test_Y):
  model.fit(train_X,train_Y.ravel())
  predictions = model.predict(test_X)
  probs = model.predict_proba(test_X)
  print(classification_report(test_Y, predictions))
  print(confusion_matrix(test_Y, predictions))
  print(accuracy_score(test_Y, predictions))
  print(matthews_corrcoef(test_Y, predictions))
  return predictions, probs


In [ ]:
def test_on_model(model,test_X,test_Y):
  predictions = model.predict(test_X)
  probs = model.predict_proba(test_X)
  print(classification_report(test_Y, predictions))
  print(confusion_matrix(test_Y, predictions))
  print(accuracy_score(test_Y, predictions))
  print(matthews_corrcoef(test_Y, predictions))
  return predictions, probs


In [ ]:
def get_mcc_curve(y_true, predictions_prob):
  cutoffs = np.arange(0,1,1e-4)
  mccs = []
  for cutoff in cutoffs:
    predictions = labels = (predictions_prob > cutoff).astype(np.int)
    mccs.append(matthews_corrcoef(y_true, predictions))
  return cutoffs, mccs



In [ ]:
import matplotlib
## plot ROC curve
def plot_roc_curve(y_true, y_probs, title):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, _ = roc_curve(y_true, y_probs, pos_label=1)
    roc_auc = auc(fpr, tpr)
    matplotlib.rc('font', size=20)
    fig = plt.figure(figsize=(8, 8))
    lw = 2
    plt.plot(fpr, tpr, color='black',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.legend(loc="lower right")
    fig.tight_layout()
    fig.savefig("./Figures/" + title + ".pdf", bbox_inches='tight')
    plt.show()


## plot MCC curve
def plot_mcc_curve(y_true, y_probs,title ):
  cuttofs, mccs = get_mcc_curve(y_true,y_probs)
  matplotlib.rc('font', size=20)
  fig = plt.figure(figsize=(8, 8))
  lw = 2
  plt.plot(cuttofs, mccs, color='black',
            lw=lw)
  #plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 0.41])
  plt.xlabel('Toxicity Probability Cutoff')
  plt.ylabel('MCC')
  #plt.legend(loc="lower right")
  fig.tight_layout()
  fig.savefig("./Figures/" + title + ".pdf", bbox_inches='tight')
  plt.show()

## plot Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, classes, title,
                          normalize=False, cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    # classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    matplotlib.rc('font', size=20)
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111)

    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           ylabel='True label',
           xlabel='Predicted label')
    _ = plt.xlabel("Predicted Labels", fontsize=18)
    _ = plt.ylabel("True label", fontsize=18)

    plt.rc('xtick', labelsize=14)
    plt.rc('ytick', labelsize=14)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center", fontsize=14,
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    fig.savefig("./Figures/" + title + ".pdf", bbox_inches='tight')
    return ax

In [ ]:
train_latents = np.array(train_latents).reshape(-1,64)#
test_latents = np.array(test_latents).reshape(-1,64)

In [ ]:
## Parameters for performing grid searches
params = {
        'boosting_type': 'dart',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'feature_fraction': 0.85,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    }

param_grid ={'boosting_type': ['dart','lgbm'],
             'objective': ['binary'],
             'n_estimators': [1200,1400,1600,1800],
             'learning_rate': [0.1,0.05],
             'feature_fraction': [0.9,1.0],
             'num_leaves' :[31,63,127,255,511,800],
             'lambda_l2 ':[0.0,0.1,0.5,1,5]
             }

In [ ]:
# make an ensemble prediction for classification
def ensemble_probabilities(members, testX):
	# make predictions
	yhats = [model.predict_proba(testX)[:,1] for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = summed/len(members)
	return np.reshape(result,(len(result),1))
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = summed/len(members)
	return np.reshape(result,(len(result),1))

In [ ]:
from sklearn import metrics

def TPR(y_true, y_pred):
    # counts the number of true positives (y_true = 1, y_pred = 1)
    tp = list((y_true == 1) & (y_pred == 1)).count(True)
    n = list((y_true == 1)).count(True)
    return tp/n
def FNR(y_true, y_pred):
    # counts the number of false negatives (y_true = 1, y_pred = 0)
    fn = list((y_true == 1) & (y_pred == 0)).count(True)
    p = list(y_true == 1).count(True)
    return fn/p
def FPR(y_true, y_pred):
    # counts the number of false positives (y_true = 0, y_pred = 1)
    fp = list((y_true == 0) & (y_pred == 1)).count(True)
    n = list(y_true == 0).count(True)
    return fp/n
def TNR(y_true, y_pred):
    # counts the number of true negatives (y_true = 0, y_pred = 0)
    tn= list((y_true == 0) & (y_pred == 0)).count(True)
    n = list(y_true == 0).count(True)
    return tn/n

In [ ]:
## Found parameters for the DART and LGBM models
params_lgbm = {
        'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'feature_fraction': 1.0,
    'num_leaves':63,
    'verbose': 1,
    'min_data_in_leaf':10,
    ' n_estimators' : 2000,
    }
  

params_dart = {
      'boosting_type': 'dart',
  'objective': 'binary',
  'metric': 'binary_logloss',
  'learning_rate': 0.05,
  'feature_fraction': 1.0,
  'num_leaves':127,
  'verbose': 1,
  'min_data_in_leaf':10,
  ' n_estimators' : 2000,
  }

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
param_grid ={'max_depth': [10, 20, 30, 40, 50,60,70,80,90,100],
             'n_estimators': [100,200, 400, 600, 800, 1000, 1200]}
     

In [ ]:
import tensorflow.keras as keras
def get_nnmodel():
  my_init = keras.initializers.glorot_uniform(seed=1)
  model = keras.models.Sequential()
  model.add(keras.layers.Dense(units=4096, input_dim=64,
    activation='relu', kernel_initializer=my_init)) 
  model.add(keras.layers.Dropout(0.7))
  model.add(keras.layers.Dense(units=2048, activation='relu',
    kernel_initializer=my_init)) 
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(units=1024, activation='relu',
    kernel_initializer=my_init))
  model.add(keras.layers.Dropout(0.5)) 
  model.add(keras.layers.Dense(units=512, activation='relu',
    kernel_initializer=my_init)) 
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(units=256, activation='relu',
    kernel_initializer=my_init)) 
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(units=512, activation='relu',
    kernel_initializer=my_init))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(units=1024, activation='relu',
    kernel_initializer=my_init)) 
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(units=1, activation='sigmoid',
      kernel_initializer=my_init))
  simple_sgd = keras.optimizers.SGD(lr=0.1)  
  model.compile(loss='binary_crossentropy',
    optimizer=simple_sgd,  metrics=['accuracy'])
  return model

In [ ]:
def fit_QDAmodel(train_X, train_Y):
  global random_state;
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  random_state= random_state +1
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y.ravel())
  y_rs = np.array(y_rs)
  x_rs = np.array(x_rs)
  QDA = QuadraticDiscriminantAnalysis()
  QDA.fit(x_rs,y_rs)
  return QDA

In [ ]:
def fit_LGBmodel(train_X, train_Y,test_X,test_Y):
  global random_state;
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  random_state= random_state +1
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y)
  lgb_train = lgb.Dataset(x_rs, y_rs)
  lgb_eval = lgb.Dataset(test_X, test_Y.flatten(), reference=lgb_train)
  model = lgb.train(params_lgbm, 
                    lgb_train, 
                    num_boost_round=2000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=100,
                    verbose_eval=True)
  return model

In [ ]:
def fit_DARTmodel(train_X, train_Y,test_X,test_Y):
  global random_state;
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  random_state= random_state +1
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y)
  lgb_train = lgb.Dataset(x_rs, y_rs)
  lgb_eval = lgb.Dataset(test_X, test_Y.flatten(), reference=lgb_train)
  model = lgb.train(params_dart, 
                    lgb_train, 
                    num_boost_round=100,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=10,
                    verbose_eval=True)
  return model

In [ ]:
def fit_SVMmodel(train_X, train_Y):
  global random_state;
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  random_state= random_state +1
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y.ravel())
  y_rs = np.array(y_rs)
  x_rs = np.array(x_rs)
  SVM = SVC(probability = True)
  SVM.fit(x_rs,y_rs)
  return SVM

In [ ]:
def fit_LDAmodel(train_X, train_Y):
  global random_state;
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  random_state= random_state +1
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y.ravel())
  y_rs = np.array(y_rs)
  x_rs = np.array(x_rs)
  LDA = LinearDiscriminantAnalysis()
  LDA.fit(x_rs,y_rs)
  return LDA

In [ ]:
from imblearn.over_sampling import RandomOverSampler
def fit_nnmodel(train_X, train_Y,test_X,test_Y):
  global random_state;
  random_state= random_state +1
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y)
  model = get_nnmodel()
  model.fit(x_rs, 
            y_rs,
            batch_size=128,
            shuffle=True, 
            validation_data=(test_X,test_Y),
            epochs=10, 
            verbose=0)
  return model

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
def fit_etmodel(train_X, train_Y):
  global random_state;
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  random_state= random_state +1
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y)
  etc = ExtraTreesClassifier(n_estimators=1000,max_depth=50)
  etc.fit(train_X,train_Y)
  return etc

In [ ]:
def fit_rfmodel(train_X, train_Y):
  global random_state;
  sampler = RandomUnderSampler(ratio={0: MIN  , 1:MIN },random_state=random_state)
  random_state= random_state +1
  x_rs, y_rs = sampler.fit_sample(train_X, train_Y)
  random_forest = RandomForestClassifier(n_estimators=1000,max_depth=30)
  random_forest.fit(train_X,train_Y)
  return random_forest

In [ ]:
n_members = 1
random_state= 0 
members_rf = [fit_rfmodel(train_latents, train_Y) for _ in range(n_members)]

In [ ]:
predictions_rf = ensemble_predictions(members_rf,test_latents)

In [ ]:
n_members = 10
random_state= 0 
members_nn = [fit_nnmodel(train_latents, train_Y,test_latents,test_Y) for _ in range(n_members)]
predictions_nn = ensemble_predictions(members_nn,test_latents)

In [ ]:
n_members = 10
random_state= 0 
members_lda = [fit_LDAmodel(train_latents, train_Y) for _ in range(n_members)]
predictions_lda = ensemble_predictions(members_lda,test_latents)

In [ ]:
n_members = 10
random_state= 0 
members_qda = [fit_QDAmodel(train_latents, train_Y) for _ in range(n_members)]
predictions_qda = ensemble_predictions(members_qda,test_latents)

In [ ]:
n_members = 10
random_state = 0
members_dart =[fit_DARTmodel(train_latents, train_Y,test_latents,test_Y) for _ in range(n_members)]
predictions_dart = ensemble_predictions(members_dart,test_latents)

In [ ]:
n_members = 10
random_state = 0
members_lgbm =[fit_LGBmodel(train_latents, train_Y,test_latents,test_Y) for _ in range(n_members)]
predictions_lgbm = ensemble_predictions(members_lgbm,test_latents)

In [ ]:
n_members = 10
random_state= 0 
members_svm = [fit_SVMmodel(train_latents, train_Y) for _ in range(n_members)]
predictions_svm= ensemble_predictions(members_svm,test_latents)

In [ ]:
n_members = 1
random_state= 0 
members_etc = [fit_etmodel(train_latents, train_Y) for _ in range(n_members)]
predictions_etc = ensemble_predictions(members_etc,test_latents)

In [ ]:
all_models  = []
all_models.append(members_lgbm)
all_models.append(members_dart)
all_models.append(members_qda)
all_models.append(members_rf)
all_models.append(members_nn)
all_models.append(members_lda)
all_models.append(members_svm)
all_models.append(members_etc)

In [ ]:
all_probs = []
all_probs.append(predictions_lgbm)
all_probs.append(predictions_dart)
all_probs.append(predictions_qda)
all_probs.append(predictions_rf)
all_probs.append(predictions_nn)
#ll_probs.append(predictions_lda)
all_probs.append(predictions_svm)
all_probs.append(predictions_etc)

In [ ]:
# calculated a weighted sum of predictions
def weighted_sum(weights, yhats):
	rows = list()
	for j in range(yhats.shape[1]):
		# enumerate values
		row = list()
		for k in range(yhats.shape[2]):
			# enumerate members
			value = 0.0
			for i in range(yhats.shape[0]):
				value += weights[i] * yhats[i,j,k]
			row.append(value)
		rows.append(row)
	return array(rows)

# make an ensemble prediction for binary classification
def ensemble_predictions2(weights,all_probs):
	# make predictions
	#yhats = [model.predict(testX) for model in members]
	yhats = all_probs 
	yhats = np.array(yhats)
	# weighted sum across ensemble members
	summed = np.tensordot(yhats, weights, axes=((0),(0)))
	# argmax across classes
	result = summed
	return result


# evaluate a specific number of members in an ensemble
def evaluate_ensemble(weights,test_Y,all_probs):
	# make prediction
	yhat = ensemble_predictions2(weights,all_probs)
	labels = (yhat > 0.5).astype(np.int)
	fpr, tpr, _ = roc_curve(np.array(test_Y).flatten(), np.array(yhat).flatten(), pos_label=1)
	roc_auc = auc(fpr, tpr)
	# calculate accuracy
	#r2(np.array(test_Y).flatten(),np.array(yhat).flatten())
	return roc_auc

def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result

# grid search weights
def grid_search(test_Y,all_probs):
	# define weights to consider
	w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
	best_score, best_weights = 0.0, None
	# iterate all possible combinations (cartesian product)
	for weights in product(w, repeat=len(all_probs)):
		# skip if all weights are equal
		if len(set(weights)) == 1:
			continue
		# hack, normalize weight vector
		weights = normalize(weights)
		# evaluate weights
		score = evaluate_ensemble(weights,test_Y,all_probs)
		if score > best_score:
			best_score, best_weights = score, weights
			print('>%s %.3f' % (best_weights, best_score))
	return list(best_weights)
 

# grid search for coefficients in a weighted average ensemble for the blobs problem
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product
# grid search weights
weights = grid_search(test_Y,all_probs)
score = evaluate_ensemble(weights,test_Y,all_probs)
print('Grid Search Weights: %s, Score: %.3f' % (weights, score))